In [1]:
import os
import re
import sys
from time import time
import pandas as pd
from torch.utils.data import DataLoader
from pykospacing import spacing
from transformers import BertTokenizer
from konlpy.tag import Mecab
# from googletrans import LANGUAGES, Translator

sys.path.insert(0, '../')
from dataset import REDataset, COLUMNS, load_data
from tokenization import load_tokenizer
from models import load_model
from tokenization import SpecialToken as ST
from config import PreTrainedType, Config, ModelType, PreProcessType

In [2]:
dataset = REDataset(device='cpu')

Load raw data...	preprocessing for 'Base'...	done!
Load Tokenizer...	done!
Apply Tokenization...	done!


In [14]:
data = pd.read_csv('../preprocessed/all_csv.tsv', sep='\t', header=None)
data.columns = COLUMNS

In [ ]:
tokenizer = BertTokenizer.from_pretrained(PreTrainedType.MultiLingual)

TEMP_E1 = '철수'
TEMP_E2 = '영희'
SPACE = ' '
NOSPACE = ''

In [26]:
data['relation_state'] = data.apply(lambda x: x['relation_state'].replace(x['e1'], TEMP_E1), axis=1)
data['relation_state'] = data.apply(lambda x: x['relation_state'].replace(x['e2'], TEMP_E2), axis=1)
data['relation_state'] = data['relation_state'].apply(lambda x: x.replace(SPACE, NOSPACE))

In [27]:
%%time
data['relation_state'] = data['relation_state'].apply(lambda x: spacing(x))

,id,relation_state,e1,e1_start,e1_end,e2,e2_start,e2_end,label
0,from train_csv 5,"카터는영희와이스라엘을조정하여,캠프데이비드에서철수대통령과메나헴베긴수상과함께중동평화를위...",안와르 사다트,38,44,이집트,5,7,인물:출신성분/국적
1,from train_csv 8,"카터는이집트와영희을조정하여,캠프데이비드에서안와르사다트대통령과메나헴베긴수상과함께중동평...",캠프데이비드 협정,78,86,이스라엘,11,14,단체:구성원
2,from train_csv 9,"카터는영희와이스라엘을조정하여,캠프데이비드에서안와르사다트대통령과메나헴베긴수상과함께중동...",캠프데이비드 협정,78,86,이집트,5,7,단체:구성원
3,from train_csv 11,"카터는이집트와영희을조정하여,캠프데이비드에서안와르사다트대통령과철수수상과함께중동평화를위...",메나헴 베긴,52,57,이스라엘,11,14,인물:출신성분/국적
4,from train_csv 65,선거전까지각축전을벌인유력한후보는15대대통령선거에나와김대중에게패배한제1야당철수의대표'...,한나라당,63,66,이회창,75,77,단체:구성원
...,...,...,...,...,...,...,...,...,...
229606,from dev_csv 99215,"2.""영희""(철수작사/계동균작곡)–03:24",박건호,11,13,사슴여인,4,7,인물:제작
229607,from dev_csv 99216,"9.""영희""(철수작사/철수작곡)-02:06",장덕,14,15,안녕히 계세요,4,10,인물:제작
229608,from dev_csv 99217,"2009년,블런트는장마크발레가감독을맡고철수가각본을쓴《영희》에서빅토리아여왕역으로출연하였다.",줄리언 펠로스,27,33,영 빅토리아,43,48,인물:제작
229609,from dev_csv 99218,2008년저드애퍼토가제작하고세스로건과철수가각본을맡은코미디영화《영희》와무술영화《겟썸》...,에번 골드버그,26,32,파인애플 익스프레스,50,59,인물:제작
